In [ ]:
NAME = "Junyu Zhu"

In [ ]:
#I use generated fake data instead to show/test the project since notebook lacks some module

In [ ]:
import pandas as pd
import random
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
#from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score
from scipy.stats import spearmanr
from scipy.stats import entropy
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Download necessary NLTK resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')


In [ ]:
# Section 1: Getting data(Not used here)

In [ ]:
# Read posts data
posts_df = pd.read_csv('sample_posts.txt', sep='\t')
print("Posts Data:")
print(posts_df.head())

# Read survey data
survey_df = pd.read_csv('sample_survey.txt', sep='\t')
print("\nSurvey Data:")
print(survey_df.head())

In [ ]:
# Section 1: Data Generation

In [ ]:
# Generate Fake Data
def generate_fake_data(platform, num_samples):
    fake_data = []
    for _ in range(num_samples):
        fake_data.append({
            "platform": platform,
            "text": f"This is a sample post about {random.choice(['topic one', 'topic two', 'topic three', 'topic four', 'topic five'])}.",
            "created_at": pd.Timestamp.now() - pd.to_timedelta(random.randint(1, 1000), unit='h'),
            "user_id": random.randint(1000, 9999)
        })
    return pd.DataFrame(fake_data)

# Combine fake data from multiple platforms
def create_fake_dataset():
    platforms = ["Twitter", "Facebook", "Instagram", "Bluesky", "Mastodon"]
    data_frames = [generate_fake_data(platform, 100) for platform in platforms]
    return pd.concat(data_frames, ignore_index=True)


In [ ]:
# Generate the dataset and print the first few rows
data = create_fake_dataset()
print("Generated Dataset (First 5 Rows):")
print(data.head())

In [ ]:
# Generate Fake Survey Data
def generate_survey_data(num_responses=100):
    survey_data = []
    for _ in range(num_responses):
        response = {
            "user_id": random.randint(1000, 9999),
            "platform": random.choice(["Twitter", "Facebook", "Instagram", "Bluesky", "Mastodon"]),
            "civility_score": random.uniform(1, 5),
            "usability_score": random.uniform(1, 5),
            "echo_chamber_perception": random.choice([True, False]),
            "intelligence": random.uniform(80, 160)  # IQ-like metric
        }
        survey_data.append(response)
    return pd.DataFrame(survey_data)


In [ ]:
# Generate the dataset and print the first few rows
survey_data = generate_survey_data()
print("Generated Dataset (First 5 Rows):")
print(survey_data.head())

In [ ]:
# Section 2: Data Cleaning and Preprocessing

In [ ]:
# Clean and preprocess text
def clean_text(text):
    text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)
    text = re.sub(r"@\w+|\#\w+", "", text)
    text = re.sub(r"[^A-Za-z\s]", "", text)
    text = text.lower()
    return text


In [ ]:
def preprocess_text(text):
    stop_words = set(stopwords.words("english"))
    tokens = word_tokenize(text)
    return " ".join([word for word in tokens if word not in stop_words])


In [ ]:
# Clean and preprocess the data
data["clean_text"] = data["text"].apply(clean_text).apply(preprocess_text)
print("\nCleaned and Preprocessed Text (First 5 Rows):")
print(data[["text", "clean_text"]].head())

In [ ]:
# Section 3: Sentiment Analysis(Show only Don't run)

In [ ]:
# Initialize VADER Sentiment Analyzer
analyzer = SentimentIntensityAnalyzer()

# Define a function to compute civility and intelligence scores
def evaluate_via_vader(text):
    """
    Evaluate civility and intelligence using VADER sentiment scores.
    Civility Score: Based on positive and negative sentiment.
    Intelligence Score: Based on the absolute compound score.
    """
    sentiment = analyzer.polarity_scores(text)
    civility_score = max(0, min(5, sentiment["pos"] * 5 - sentiment["neg"] * 5))  # Scale civility between 0-5
    intelligence_score = max(0, min(5, abs(sentiment["compound"]) * 5))  # Scale intelligence between 0-5
    return civility_score, intelligence_score

# Apply the function to the clean_text column
data["civility_score"], data["intelligence_score"] = zip(
    *data["clean_text"].apply(evaluate_via_vader)
)

In [ ]:
# Display a preview of the updated DataFrame
print("\nUpdated Data with Civility and Intelligence Scores (First 5 Rows):")
print(data[["clean_text", "civility_score", "intelligence_score"]].head())

In [ ]:
# Section 3: Sentiment Generation

In [ ]:
# Define functions to generate fake civility and intelligence scores
def generate_fake_civility():
    """Generate a fake civility score between 1 and 5."""
    return round(random.uniform(1, 5), 2)

def generate_fake_intelligence():
    """Generate a fake intelligence score between 1 and 5."""
    return round(random.uniform(1, 5), 2)

# Apply the functions to generate scores
data["civility_score"] = data["clean_text"].apply(lambda x: generate_fake_civility())
data["intelligence_score"] = data["clean_text"].apply(lambda x: generate_fake_intelligence())


In [ ]:
# Display a preview of the updated DataFrame
print("\nUpdated Data with Fake Civility and Intelligence Scores (First 5 Rows):")
print(data[["clean_text", "civility_score", "intelligence_score"]].head())

In [ ]:
# Section 4: Clustering and Echo Chamber Detection

In [ ]:
# Detect Echo Chambers/Clustering
def detect_echo_chambers(texts, num_clusters=3):
    #Detect echo chambers by clustering similar content.
    #texts: List of text data to cluster.
    #num_clusters: Number of clusters to form.
    #Returns labels: Cluster labels for each text entry.
    
    # Step 1: Vectorize text using TF-IDF
    vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
    X = vectorizer.fit_transform(texts)
    # Step 2: Apply K-Means clustering
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    labels = kmeans.fit_predict(X)
    return labels


In [ ]:
# Apply clustering for echo chamber detection
data["cluster"] = detect_echo_chambers(data["clean_text"])
print("\nClustering Results (First 5 Rows):")
print(data[["clean_text", "cluster"]].head())


In [ ]:
# Attempt: Visualize clusters (Dimensionality Reduction with PCA)
from sklearn.decomposition import PCA

def plot_clusters(data, texts, cluster_column):
    """
    Visualize the clustering results.

    Parameters:
    - data: DataFrame containing the data and cluster assignments.
    - texts: Original text data (used for dimensionality reduction).
    - cluster_column: Column name indicating cluster labels.
    """
    vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
    X = vectorizer.fit_transform(texts)

    # Reduce dimensions for visualization
    pca = PCA(n_components=2)
    reduced_data = pca.fit_transform(X.toarray())

    plt.figure(figsize=(10, 6))
    plt.scatter(
        reduced_data[:, 0], 
        reduced_data[:, 1], 
        c=data[cluster_column], 
        cmap='viridis', 
        s=10
    )
    plt.colorbar(label="Cluster")
    plt.title("Echo Chamber Clusters")
    plt.xlabel("PCA Component 1")
    plt.ylabel("PCA Component 2")
    plt.show()

# Plot the clusters
plot_clusters(data, data["clean_text"], "cluster")

In [ ]:
# Section 5: Algorithm Simulation

In [ ]:
# Simulate content recommendation algorithms
# Detect algorithm toxicity (frequency of controversial content recommendations)
def calculate_algorithm_toxicity(data):
    toxicity = data.groupby("platform")["civility_score"].apply(
        lambda scores: sum(score < 2.5 for score in scores) / len(scores)
    )
    toxicity_table = pd.DataFrame({"platform": toxicity.index, "algorithm_toxicity": toxicity.values})
    return toxicity_table

# Create algorithm toxicity table
toxicity_table = calculate_algorithm_toxicity(data)
print("\nAlgorithm Toxicity Table:")
print(toxicity_table)

# Graph algorithm toxicity by platform
plt.figure(figsize=(10, 6))
sns.barplot(x="platform", y="algorithm_toxicity", data=toxicity_table, palette="coolwarm")
plt.title("Algorithm Toxicity by Platform (Controversial Content Frequency)")
plt.xlabel("Platform")
plt.ylabel("Algorithm Toxicity (Frequency)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# Section 6: Database Integration

In [ ]:
# Database setup
def setup_database():
    conn = sqlite3.connect("social_media_analysis.db")
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS posts (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            platform TEXT,
            text TEXT,
            clean_text TEXT,
            created_at TEXT,
            user_id TEXT,
            civility_score REAL,
            intelligence_score REAL,
            cluster INTEGER
        )
    """)
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS survey (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_id TEXT,
            platform TEXT,
            civility_score REAL,
            usability_score REAL,
            echo_chamber_perception BOOLEAN,
            intelligence REAL
        )
    """)
    conn.commit()
    return conn


In [ ]:
# Insert data into the database
def store_data(conn, dataframe, table_name):
    dataframe.to_sql(table_name, conn, if_exists="append", index=False)

# Setup database and insert generated data
conn = setup_database()
store_data(conn, data, "posts")
store_data(conn, survey_data, "survey")

# Verify the data insertion by querying the tables
cursor = conn.cursor()
cursor.execute("SELECT * FROM posts LIMIT 5")
print("Posts Table (First 5 Rows):")
for row in cursor.fetchall():
    print(row)

cursor.execute("SELECT * FROM survey LIMIT 5")
print("Survey Table (First 5 Rows):")
for row in cursor.fetchall():
    print(row)

# Close the connection
#conn.close()

In [ ]:
# Section 7: Queries/Database(sql) manipulations

In [ ]:
# Query execution
def run_query(query):
    conn = sqlite3.connect("social_media_analysis.db")
    cursor = conn.cursor()
    cursor.execute(query)
    results = cursor.fetchall()
    conn.close()
    return results


In [ ]:
print("Analyzing relationships with SQL queries...")

query = """
SELECT 
    p.platform AS platform,
    p.cluster AS cluster,
    COUNT(p.id) AS post_count
FROM posts p
GROUP BY p.platform, p.cluster
ORDER BY platform, cluster;
"""
result = pd.read_sql_query(query, conn)
print("\nCluster Distribution Across Platforms:")
print(result)


In [ ]:
query = """
SELECT 
    p.platform AS platform,
    AVG(p.civility_score) AS avg_civility,
    AVG(p.intelligence_score) AS avg_intelligence
FROM posts p
GROUP BY p.platform
ORDER BY avg_civility DESC;
"""
result = pd.read_sql_query(query, conn)
print("\nRelationships Between Civility and Intelligence Levels by Platform:")
print(result)


In [ ]:
query = """
SELECT 
    p.cluster AS cluster,
    AVG(p.civility_score) AS avg_civility,
    AVG(p.intelligence_score) AS avg_intelligence,
    COUNT(p.id) AS post_count
FROM posts p
GROUP BY p.cluster
ORDER BY avg_civility DESC, avg_intelligence DESC;
"""
result = pd.read_sql_query(query, conn)
print("\nCivility and Intelligence Levels by Clusters:")
print(result)


In [ ]:
query = """
SELECT 
    p.id AS post_id,
    p.platform AS post_platform,
    p.text AS post_text,
    s.platform AS survey_platform,
    s.civility_score AS survey_civility,
    s.usability_score
FROM posts p
JOIN survey s ON p.user_id = s.user_id
LIMIT 10;
"""
result = pd.read_sql_query(query, conn)
print("\nPosts with Survey Data (First 10 Rows):")
print(result)


In [ ]:
query = """
SELECT 
    p.platform AS post_platform,
    AVG(p.civility_score) AS avg_post_civility,
    AVG(s.civility_score) AS avg_survey_civility
FROM posts p
JOIN survey s ON p.user_id = s.user_id
GROUP BY p.platform;
"""
result = pd.read_sql_query(query, conn)
print("\nAverage Civility by Platform:")
print(result)


In [ ]:
query = """
SELECT 
    AVG(s.usability_score) AS avg_usability,
    AVG(p.civility_score) AS avg_post_civility
FROM survey s
JOIN posts p ON s.user_id = p.user_id
GROUP BY s.platform;
"""
result = pd.read_sql_query(query, conn)
print("\nUsability vs. Civility by Platform:")
print(result)


In [ ]:
query = """
SELECT 
    p.id AS post_id,
    p.civility_score AS post_civility,
    s.civility_score AS survey_civility,
    p.clean_text
FROM posts p
JOIN survey s ON p.user_id = s.user_id
WHERE p.civility_score < 2
ORDER BY p.civility_score ASC
LIMIT 10;
"""
result = pd.read_sql_query(query, conn)
print("\nComparison of Controversial Posts and Survey Civility (First 10 Rows):")
print(result)


In [ ]:
# close database
conn.close()

In [ ]:
# Section 8: Visualization and Data Analysis

In [ ]:
# Civility Distribution
sns.histplot(data["civility_score"], bins=5, kde=False)
plt.title("Distribution of Civility Scores")
plt.xlabel("Civility Score")
plt.ylabel("Frequency")
plt.show()

# Intelligence vs Civility
sns.scatterplot(x="civility_score", y="intelligence_score", hue="cluster", data=data)
plt.title("Civility vs Intelligence by Cluster")
plt.xlabel("Civility Score")
plt.ylabel("Intelligence Score")
plt.legend(title="Cluster")
plt.show()

In [ ]:
# Cluster Distribution Across Platforms
cluster_distribution = data.groupby(["platform", "cluster"]).size().reset_index(name="count")
total_counts = cluster_distribution.groupby("platform")["count"].transform("sum")
cluster_distribution["proportion"] = cluster_distribution["count"] / total_counts

print("\nCluster Distribution Across Platforms (Proportion):")
print(cluster_distribution)

# Visualization: Stacked Bar Chart for Cluster Distribution
platforms = cluster_distribution["platform"].unique()
clusters = cluster_distribution["cluster"].unique()
cluster_pivot = cluster_distribution.pivot(index="platform", columns="cluster", values="proportion").fillna(0)

cluster_pivot.plot(kind="bar", stacked=True, figsize=(10, 6), colormap="viridis")
plt.title("Cluster Distribution Across Platforms (Proportion)")
plt.xlabel("Platform")
plt.ylabel("Proportion of Clusters")
plt.legend(title="Cluster", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()

In [ ]:
# Intelligence Scores Across Platforms
intelligence = data.groupby("platform")["intelligence_score"].mean().reset_index()
print("\nIntelligence Scores Across Platforms:")
print(intelligence)

# Visualization: Intelligence Scores
plt.figure(figsize=(10, 6))
sns.barplot(x="platform", y="intelligence_score", data=intelligence, palette="coolwarm")
plt.title("Intelligence Scores Across Platforms")
plt.xlabel("Platform")
plt.ylabel("Average Intelligence Score")
plt.show()

In [ ]:
import numpy as np
# Perception vs Reality (Civility and Intelligence)
# Aggregating actual scores from the posts
actual_scores = data.groupby("platform").agg({
    "civility_score": "mean",
    "intelligence_score": "mean"
}).reset_index()
actual_scores.columns = ["platform", "actual_civility_score", "actual_intelligence_score"]

# Aggregating survey scores
survey_scores = survey_data.groupby("platform").agg({
    "civility_score": "mean",
    "intelligence": "mean"
}).reset_index()
survey_scores.columns = ["platform", "perceived_civility_score", "perceived_intelligence_score"]

# Merge the two datasets for comparison
perception_vs_reality = pd.merge(actual_scores, survey_scores, on="platform")

print("\nPerception vs Reality of Civility and Intelligence Scores:")
print(perception_vs_reality)

# Visualization: Grouped Bar Chart for Perception vs Reality
fig, ax = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

x = np.arange(len(perception_vs_reality["platform"]))
width = 0.35  # Bar width

# Civility Comparison
ax[0].bar(x - width/2, perception_vs_reality["actual_civility_score"], width, label="Actual Civility Score", color="skyblue")
ax[0].bar(x + width/2, perception_vs_reality["perceived_civility_score"], width, label="Perceived Civility Score", color="orange")
ax[0].set_title("Perception vs Reality of Civility Scores Across Platforms")
ax[0].set_ylabel("Civility Score")
ax[0].set_xticks(x)
ax[0].set_xticklabels(perception_vs_reality["platform"])
ax[0].legend()

# Intelligence Comparison
ax[1].bar(x - width/2, perception_vs_reality["actual_intelligence_score"], width, label="Actual Intelligence Score", color="lightgreen")
ax[1].bar(x + width/2, perception_vs_reality["perceived_intelligence_score"], width, label="Perceived Intelligence Score", color="purple")
ax[1].set_title("Perception vs Reality of Intelligence Scores Across Platforms")
ax[1].set_ylabel("Intelligence Score")
ax[1].set_xticks(x)
ax[1].set_xticklabels(perception_vs_reality["platform"])
ax[1].legend()

plt.xlabel("Platform")
plt.tight_layout()
plt.show()


In [ ]:
# Overall User Environment Score Calculation
# Calculate overall user environment score with adjusted weights
def calculate_overall_environment_weighted(data, survey_data):
    # Aggregate civility, intelligence, and cluster information
    post_metrics = data.groupby("platform").agg({
        "civility_score": "mean",
        "intelligence_score": "mean",
        "cluster": "nunique"
    }).reset_index()
    post_metrics.columns = ["platform", "avg_civility_score", "avg_intelligence_score", "num_clusters"]

    # Add usability from survey data
    survey_metrics = survey_data.groupby("platform")["usability_score"].mean().reset_index()
    survey_metrics.columns = ["platform", "avg_usability_score"]

    # Combine metrics
    overall_metrics = pd.merge(post_metrics, survey_metrics, on="platform")
    
    # Assign weights: civility and intelligence (0.4 each), usability and clusters (0.1 each), algorithm toxicity (0.2)
    civility_weight = 0.4
    intelligence_weight = 0.4
    usability_weight = 0.1
    cluster_weight = 0.1
    algorithm_weight = 0.2

    overall_metrics["overall_score"] = (
        (overall_metrics["avg_civility_score"] * civility_weight) +
        (overall_metrics["avg_intelligence_score"] * intelligence_weight) +
        (overall_metrics["avg_usability_score"] * usability_weight) -
        (toxicity_table["algorithm_toxicity"] * algorithm_weight) +
        ((5 - overall_metrics["num_clusters"]) * cluster_weight)  # Inverse cluster count for echo chamber score
    )
    return overall_metrics

# Calculate and display results
overall_metrics_weighted = calculate_overall_environment_weighted(data, survey_data)

# Graph overall user environment with emphasized app name and score
def plot_weighted_overall_environment(overall_metrics):
    plt.figure(figsize=(10, 6))
    plt.axis('off')  # Turn off axes for emphasis on the labels
    
    # Create a custom text-based display
    for i, row in enumerate(overall_metrics.itertuples()):
        app_text = f"**{row.platform}**"
        score_text = f"{row.overall_score:.2f}"
        plt.text(0.5, 1 - i * 0.15, app_text, fontsize=14, ha="center", color="red", weight="bold")
        plt.text(0.5, 1 - i * 0.15 - 0.03, score_text, fontsize=12, ha="center", color="black")
    
    plt.tight_layout()

# Call the function to plot
plot_weighted_overall_environment(overall_metrics_weighted)

In [ ]:
# Section 9: Machine Learning: Relationships and Analysis

In [ ]:
# Civility and Intelligence
def analyze_relationships(data):
    # Use Linear Regression to analyze relationships.
    # Civility and Intelligence Relationship
    X = data[["civility_score"]]
    y = data["intelligence_score"]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    
    print(f"Mean Squared Error: {mse:.2f}")
    print(f"R-squared: {r2:.2f}")
    return model

relationship_model = analyze_relationships(data)


In [ ]:
# Algorithms and Echo chambers
# Use linear regression to analyze the relationship between algorithm toxicity and echo chambers
from sklearn.linear_model import LinearRegression

def analyze_toxicity_vs_clusters(toxicity_table, data):
    cluster_data = data.groupby("platform")["cluster"].nunique().reset_index()
    cluster_data.columns = ["platform", "num_clusters"]
    combined_data = pd.merge(toxicity_table, cluster_data, on="platform")

    # Regression
    X = combined_data["algorithm_toxicity"].values.reshape(-1, 1)
    y = combined_data["num_clusters"].values
    model = LinearRegression().fit(X, y)

    # Graph the relationship
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x="algorithm_toxicity", y="num_clusters", data=combined_data, s=100, color="orange")
    plt.plot(combined_data["algorithm_toxicity"], model.predict(X), color="blue", label="Regression Line")
    plt.title("Relationship Between Algorithm Toxicity and Echo Chambers")
    plt.xlabel("Algorithm Toxicity")
    plt.ylabel("Number of Clusters (Echo Chambers)")
    plt.legend()
    plt.tight_layout()
    plt.show()

    return combined_data, model.coef_, model.intercept_

toxicity_vs_clusters_data, coef, intercept = analyze_toxicity_vs_clusters(toxicity_table, data)
print("\nCombined Data (Algorithm Toxicity vs. Echo Chambers):")
print(toxicity_vs_clusters_data)
print(f"\nLinear Regression Coefficient: {coef[0]:.2f}, Intercept: {intercept:.2f}")


In [ ]:
# Summary
print("\nFinal Data Sample:")
print(data.head())
print(survey_data.head())
print("\nOverall User Environment Scores:")
print(overall_metrics)